In [171]:
import numpy as np
import pandas as pd
import random
import tensorflow as tf

from tensorflow import keras
from keras import backend as K
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.layers import LSTM, Reshape, RepeatVector, TimeDistributed
from sklearn.model_selection import train_test_split
from keras.preprocessing.text import Tokenizer
from collections import Counter

import numpy as np
import os
from mido import MidiFile, MidiTrack, Message
import mido

In [106]:
paths = []
songs = []
for r, d, f in os.walk(r"C:\Users\Vee\Desktop\python\songs"):
    for file in f:
        if '.mid' in file:
            paths.append(os.path.join(r, file))

for path in paths:
    mid = MidiFile(path, type = 1)
    songs.append(mid)

In [124]:
notes = []
dataset = []

In [125]:
#for each in midi object in list of songs
for i in range(len(songs)):
    #for each note in midi object
    for msg in songs[i]:
        #filtering out meta messages
        if not msg.is_meta:
            #filtering out control changes
            if (msg.type == 'note_on') or (msg.type == 'note_off'):
                #normalizing note and velocity values
                notes.append([msg.note/127, msg.velocity/127, msg.time])
    #if more than 30 notes delete them
    if len(notes) > 60:
        for x in range(len(notes)-60):
            notes = np.delete(notes, 60, 0)
    #if less than 30 notes pad with zeros
    elif len(notes) < 60:
        for x in range(60 - len(notes)):
            notes.append([0,0,0])
    dataset.append(notes)    
    notes = []

In [126]:
dataset = np.array(dataset)
dataset.shape

(17, 60, 3)

In [127]:
bruv = []
#for each in midi object in list of songs
for i in range(len(songs)):
    #for each note in midi object
    for msg in songs[i]:
        #filtering out meta messages
        if not msg.is_meta:
            #filtering out control changes
            if (msg.type == 'note_on') or (msg.type == 'note_off'):
                #normalizing note and velocity values
                notes.append([msg.note/127, msg.velocity/127, msg.time])
    #if more than 40 notes delete them
    if len(notes) > 4:
        for x in range(len(notes)-4):
            notes = np.delete(notes, 4, 0)
    bruv.append(notes)    
    notes = []

In [128]:
bruv = np.array(bruv)
bruv.shape

(17, 4, 3)

In [129]:
#splitting data into train and test sets. 3/4 train, 1/4 test.
x_train,x_test,y_train,y_test = train_test_split(bruv, dataset, test_size=0.2, shuffle=False, random_state=42)

In [130]:
# define model
model = Sequential()
#shaping input to match data
model.add(LSTM(200, activation='relu', input_shape=(4, 3)))
#specifying output to have 60 timesteps
model.add(RepeatVector(60))
model.add(LSTM(200, activation='relu', return_sequences=True))
model.add(LSTM(200, activation='relu', return_sequences=True))
model.add(LSTM(200, activation='relu', return_sequences=True))
#specifying 3 features as the output
model.add(TimeDistributed(Dense(3)))
model.add(TimeDistributed(Dense(3)))
model.add(Dropout(0.2))
model.add(TimeDistributed(Dense(3)))
model.add(TimeDistributed(Dense(3)))
model.add(TimeDistributed(Dense(3)))
model.add(Activation('tanh'))
model.compile(loss='mean_absolute_error', optimizer='adam')
print(model.summary())

Model: "sequential_36"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_216 (LSTM)              (None, 200)               163200    
_________________________________________________________________
repeat_vector_6 (RepeatVecto (None, 60, 200)           0         
_________________________________________________________________
lstm_217 (LSTM)              (None, 60, 200)           320800    
_________________________________________________________________
lstm_218 (LSTM)              (None, 60, 200)           320800    
_________________________________________________________________
lstm_219 (LSTM)              (None, 60, 200)           320800    
_________________________________________________________________
time_distributed_14 (TimeDis (None, 60, 3)             603       
_________________________________________________________________
time_distributed_15 (TimeDis (None, 60, 3)           

In [131]:
model.fit(bruv, dataset, epochs=20, batch_size=5, verbose=2,validation_data=(x_test, y_test))

Train on 17 samples, validate on 4 samples
Epoch 1/20
 - 9s - loss: 0.3197 - val_loss: 0.2685
Epoch 2/20
 - 2s - loss: 0.3141 - val_loss: 0.2624
Epoch 3/20
 - 2s - loss: 0.3081 - val_loss: 0.2553
Epoch 4/20
 - 2s - loss: 0.3011 - val_loss: 0.2472
Epoch 5/20
 - 2s - loss: 0.2953 - val_loss: 0.2411
Epoch 6/20
 - 2s - loss: 0.2887 - val_loss: 0.2341
Epoch 7/20
 - 2s - loss: 0.2822 - val_loss: 0.2261
Epoch 8/20
 - 2s - loss: 0.2742 - val_loss: 0.2164
Epoch 9/20
 - 2s - loss: 0.2655 - val_loss: 0.2021
Epoch 10/20
 - 2s - loss: 0.2564 - val_loss: 0.1927
Epoch 11/20
 - 2s - loss: 0.2471 - val_loss: 0.1774
Epoch 12/20
 - 2s - loss: 0.2419 - val_loss: 0.1748
Epoch 13/20
 - 2s - loss: 0.2360 - val_loss: 0.1716
Epoch 14/20
 - 2s - loss: 0.2301 - val_loss: 0.1666
Epoch 15/20
 - 2s - loss: 0.2273 - val_loss: 0.1681
Epoch 16/20
 - 2s - loss: 0.2213 - val_loss: 0.1620
Epoch 17/20
 - 2s - loss: 0.2170 - val_loss: 0.1586
Epoch 18/20
 - 2s - loss: 0.2138 - val_loss: 0.1551
Epoch 19/20
 - 2s - loss: 0.21

In [161]:
predict = model.predict(bruv)
#disregarding negative values
predict = abs(predict)

In [163]:
#adjusting from normalization
for x in range(60):
    for i in range(2):
        predict[0][x][i] = predict[0][x][i] * 127

In [166]:
predict[0]

array([[4.07358360e+01, 1.11188946e+01, 7.49659911e-03],
       [4.35735664e+01, 1.29605055e+01, 2.74126958e-02],
       [4.65148582e+01, 1.48792925e+01, 4.68983315e-02],
       [4.93690338e+01, 1.67606735e+01, 6.48470148e-02],
       [5.19895172e+01, 1.85057411e+01, 8.03357586e-02],
       [5.42825127e+01, 2.00486965e+01, 9.28409696e-02],
       [5.62055969e+01, 2.13602562e+01, 1.02305971e-01],
       [5.77584152e+01, 2.24324493e+01, 1.08900346e-01],
       [5.89620323e+01, 2.32734833e+01, 1.12913437e-01],
       [5.98603287e+01, 2.39086304e+01, 1.14805885e-01],
       [6.04956398e+01, 2.43628864e+01, 1.14970505e-01],
       [6.09251099e+01, 2.46772938e+01, 1.14055589e-01],
       [6.11967583e+01, 2.48850040e+01, 1.12498991e-01],
       [6.13485985e+01, 2.50109520e+01, 1.10568129e-01],
       [6.14116020e+01, 2.50758457e+01, 1.08462729e-01],
       [6.14129219e+01, 2.50989132e+01, 1.06369190e-01],
       [6.13728905e+01, 2.50934772e+01, 1.04384236e-01],
       [6.13123932e+01, 2.50739

In [167]:
from keras.models import load_model

# Creates a HDF5 file 'my_model.h5'
model.save('my_model.h5')

In [183]:
midler = MidiFile()
track = MidiTrack()
midler.tracks.append(track)
track.append(Message('program_change', program=12, time=0))
for x in range(60):
    track.append(Message('note_on', note=int(predict[0][x][0]), velocity=int(predict[0][x][1]), time=int(predict[0][x][2])))
    track.append(Message('note_off', note=int(predict[0][x][0]), velocity=int(predict[0][x][1]), time=int(predict[0][x][2])))
midler.save('new_song.mid')

In [184]:
for msg in midler:
    print(msg)

program_change channel=0 program=12 time=0
note_on channel=0 note=40 velocity=11 time=0
note_off channel=0 note=40 velocity=11 time=0
note_on channel=0 note=43 velocity=12 time=0
note_off channel=0 note=43 velocity=12 time=0
note_on channel=0 note=46 velocity=14 time=0
note_off channel=0 note=46 velocity=14 time=0
note_on channel=0 note=49 velocity=16 time=0
note_off channel=0 note=49 velocity=16 time=0
note_on channel=0 note=51 velocity=18 time=0
note_off channel=0 note=51 velocity=18 time=0
note_on channel=0 note=54 velocity=20 time=0
note_off channel=0 note=54 velocity=20 time=0
note_on channel=0 note=56 velocity=21 time=0
note_off channel=0 note=56 velocity=21 time=0
note_on channel=0 note=57 velocity=22 time=0
note_off channel=0 note=57 velocity=22 time=0
note_on channel=0 note=58 velocity=23 time=0
note_off channel=0 note=58 velocity=23 time=0
note_on channel=0 note=59 velocity=23 time=0
note_off channel=0 note=59 velocity=23 time=0
note_on channel=0 note=60 velocity=24 time=0
no